# Perform Sentiment Analysis on Reddit Posts

Reddit API:

Go to https://www.reddit.com/prefs/apps
Click "create application"
Name: My Sentiment Example
Choose script
Leave description and about url blank
Add http://www.example.com/unused/redirect/uri to redirect uri

## Library Imports

In [1]:
import os
import praw
import openai

In [3]:
reddit_client_id = os.environ["REDDIT_CLIENT_ID"]
reddit_secret_key = os.environ["REDDIT_SECRET_KEY"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
'''
Initialize the Reddit API
'''
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_secret_key,
                     user_agent="Sentiment Analysis test by suvofalcon")

In [24]:
'''
Make a call to Reddit to test
'''
for post in reddit.subreddit("Stocks").hot(limit=3):
    print (f"Title - {post.title}")
    submission = reddit.submission(post.id)
    counter = 0
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]":
            print(comment)
            counter += 1
        if counter == 3:
            break
    

Title - Rate My Portfolio - r/Stocks Quarterly Thread September 2023
My [last post](https://www.reddit.com/r/stocks/comments/11ezwar/rate_my_portfolio_rstocks_quarterly_thread_march/jah0b73/) was 6 months ago.

To recap, a brief overview of my strategy, that has not since changed:

> I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much.

> I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if 